### 1) Data Preparation

In [1]:
import pandas as pd
path = "C:/Users/nchong/OneDrive - Intel Corporation/Documents/ML based speckle POC/"
df = pd.read_csv(path+"SNR_R4_ww49.1_Speckle.csv.gz")

In [ ]:
#create new column for target
# import numpy as np
# df["SPECKLE"] = np.where(df["DELTA"]==0,0,1)
# cols = df.columns.tolist()
# cols = [cols[-1]]+cols[:-1] #move speckle col to the front
# df = df.reindex(columns=cols)

In [32]:
df["DELTA"].value_counts()

0    13390
1     6317
2      129
Name: DELTA, dtype: int64

In [33]:
df["SPECKLE"].value_counts()

0    13390
1     6446
Name: SPECKLE, dtype: int64

In [4]:
df.head()

,SPECKLE,VID,ULT@MIDAS_6261_U1,SORTLOT,SORTLOT7,WAFER,XLOC,YLOC,ULT@MIDAS_6261_U2,IB@6261[CLASSHOT],...,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_SVIDCLK1,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_SVIDDATA0,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_SVIDDATA1,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TAP_DIS_STRAP_N,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TAP_ODT_EN,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TXT_AGENT,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TXT_PLTEN,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_VPPSMBUSSCL,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_VPPSMBUSSDA,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_VSENSEPMAX
0,1,M10P67G000001,N0286920_293_6_-3,N0286920,N028692,293,6,-3,H0280460_443_2_1,1,...,-8.210000e-07,-8.210000e-07,-8.330000e-07,-2.100000e-07,-2.240000e-07,-1.950000e-07,-1.910000e-07,-8.990000e-07,-8.840000e-07,-1.970000e-07
1,1,M10P67G000962,N0286920_225_5_-5,N0286920,N028692,225,5,-5,H0280460_443_2_18,1,...,-8.670000e-07,-8.870000e-07,-8.740000e-07,-2.110000e-07,-2.100000e-07,-1.900000e-07,-2.280000e-07,-8.620000e-07,-8.610000e-07,-1.920000e-07
2,1,M10P67G000808,N0292860_135_6_0,N0292860,N029286,135,6,0,H0256240_667_-10_6,1,...,-6.000000e-07,-6.500000e-07,-6.380000e-07,-1.440000e-07,-1.380000e-07,-1.440000e-07,-1.620000e-07,-5.820000e-07,-6.150000e-07,-1.460000e-07
3,1,M17U1C0300573,N0293360_159_3_1,N0293360,N029336,159,3,1,H0280460_443_0_-14,1,...,-1.170000e-06,-1.220000e-06,-1.230000e-06,-2.950000e-07,-2.930000e-07,-3.190000e-07,-3.070000e-07,-1.220000e-06,-1.190000e-06,-2.960000e-07
4,1,M10P67G000433,N0293340_045_1_5,N0293340,N029334,45,1,5,H0280460_444_9_13,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
pd.options.display.max_seq_items = 2000
df.columns

Index(['SPECKLE', 'VID', 'ULT@MIDAS_6261_U1', 'SORTLOT', 'SORTLOT7', 'WAFER',
       'XLOC', 'YLOC', 'ULT@MIDAS_6261_U2', 'IB@6261[CLASSHOT]',
       'FB@6261[CLASSHOT]',
       'ARR_SPECKLE_SCREEN::PHYTON_X_USERFUNC_K_END_X_X_X_X_PBISTSPECKLE_INDICATOR::DUTCPUBAD_1@6261[CLASSHOT]',
       'TEST RESULTS', 'TEST RESULTS BITS', 'TR_BITS', 'INCOMING',
       'INCOMING BITS', 'INC_BITS', 'OUTGOING', 'OUTGOING BITS', 'OUT_BITS',
       'DELTA', 'IDV_0001_SVT3GNES12_FULLDIE_SOC_0650_MED@132110',
       'IDV_0001_SVT3GNES12_FULLDIE_SOC_0950_MED@132110',
       'IDV_0001_SVT3GNES12_FULLDIE_SOC_TALL_0650_MED@132110',
       'IDV_0001_SVT3GNES12_FULLDIE_SOC_TALL_0950_MED@132110',
       'IDV_0002_SVT3GNES12_FULLDIE_SOC_0650_MED@132110',
       'IDV_0002_SVT3GNES12_FULLDIE_SOC_0950_MED@132110',
       'IDV_0002_SVT3GNES12_FULLDIE_SOC_TALL_0650_MED@132110',
       'IDV_0002_SVT3GNES12_FULLDIE_SOC_TALL_0950_MED@132110',
       'IDV_0003_SVT3GSEM12_FULLDIE_SOC_0650_MED@132110',
       'IDV_0003_SVT3

In [6]:
df.shape

(19836, 1615)

In [7]:
df.describe()

,SPECKLE,WAFER,XLOC,YLOC,IB@6261[CLASSHOT],FB@6261[CLASSHOT],TEST RESULTS BITS,TR_BITS,INCOMING BITS,INC_BITS,...,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_SVIDCLK1,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_SVIDDATA0,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_SVIDDATA1,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TAP_DIS_STRAP_N,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TAP_ODT_EN,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TXT_AGENT,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TXT_PLTEN,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_VPPSMBUSSCL,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_VPPSMBUSSDA,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_VSENSEPMAX
count,19836.000000,19836.000000,19836.000000,19836.000000,19836.000000,19836.000000,19836.000000,19836.000000,19836.000000,19836.000000,...,6.252000e+03,6.252000e+03,6.252000e+03,6.252000e+03,6.252000e+03,6.252000e+03,6.252000e+03,6.252000e+03,6.252000e+03,6.252000e+03
mean,0.324965,420.113329,0.377445,-0.712492,2.315084,235.525812,10625.183404,0.828594,4358.616455,0.497126,...,-8.755427e-07,-8.832898e-07,-8.893884e-07,-2.201967e-07,-2.300373e-07,-2.244919e-07,-2.219708e-07,-8.778903e-07,-8.718458e-07,-2.182973e-07
std,0.468373,261.767943,3.178078,4.817901,6.684193,680.231218,29884.293245,0.792173,19311.916064,0.514221,...,2.250728e-07,2.279325e-07,2.305911e-07,5.771449e-08,6.150872e-08,5.903851e-08,5.726272e-08,2.215545e-07,2.206803e-07,5.746509e-08
min,0.000000,1.000000,-6.000000,-10.000000,1.000000,101.000000,0.000000,0.000000,0.000000,0.000000,...,-2.200000e-06,-2.330000e-06,-2.250000e-06,-5.470000e-07,-6.690000e-07,-6.110000e-07,-5.860000e-07,-2.210000e-06,-2.230000e-06,-5.490000e-07
25%,0.000000,186.000000,-2.000000,-4.000000,1.000000,101.000000,0.000000,0.000000,0.000000,0.000000,...,-1.010000e-06,-1.020000e-06,-1.030000e-06,-2.560000e-07,-2.670000e-07,-2.600000e-07,-2.580000e-07,-1.010000e-06,-1.000000e-06,-2.520000e-07
50%,0.000000,407.000000,0.000000,-1.000000,1.000000,101.000000,1.000000,1.000000,0.000000,0.000000,...,-8.520000e-07,-8.570000e-07,-8.640000e-07,-2.150000e-07,-2.250000e-07,-2.190000e-07,-2.170000e-07,-8.545000e-07,-8.510000e-07,-2.130000e-07
75%,1.000000,624.000000,3.000000,3.000000,1.000000,101.000000,110.000000,1.000000,100.000000,1.000000,...,-7.100000e-07,-7.180000e-07,-7.207500e-07,-1.790000e-07,-1.850000e-07,-1.820000e-07,-1.800000e-07,-7.140000e-07,-7.100000e-07,-1.770000e-07
max,1.000000,999.000000,6.000000,9.000000,70.000000,7031.000000,110000.000000,2.000000,100000.000000,2.000000,...,-3.480000e-07,-3.410000e-07,-3.390000e-07,-7.130000e-08,-8.850000e-08,-7.950000e-08,-7.230000e-08,-3.550000e-07,-3.330000e-07,-7.970000e-08


### Duplicate handling

In [2]:
def duplicates(df,keep):
    """
    Check for duplicates in VID and remove duplicates if there is any
    df[DataFrame]: input df
    keep: {‘first’, ‘last’, False}: Determines which duplicates (if any) to keep. 
    - first : Drop duplicates except for the first occurrence. 
    - last : Drop duplicates except for the last occurrence. 
    - False : Drop all duplicates.
    """
    if df['VID'].duplicated().any():
        print("There are duplicates in VID")
        df_dup = df[df.duplicated(subset=['VID'])] #get the duplicates in df 
        display(df_dup)
        df = df.drop_duplicates(subset=['VID'],keep=keep) #remove duplicates from df
        return df
    else:
        print("There are no duplicates in VID")
    

In [3]:
duplicates(df,keep="first")

There are no duplicates in VID


### NA handling

In [36]:
df.shape

(19836, 1615)

In [37]:
# check column for nulls
def check_colna(df):
    """
    Check each column for nulls. Returns Feature, total null and % of null for each column
    df[DataFrame]: dataframe
    
    """
    colna_df = pd.DataFrame(columns =["Feature", "Total Null", "% of Null"])
    for col in df.columns: 
        #checking if there is any null in the column

        if df[col].isnull().sum()>0: 
            
            # if null present, total number of null in the column stores here
            total_null = df[col].isnull().sum() 
            new_row = {'Feature':col, 'Total Null':total_null, '% of Null':round(total_null*100/len(df),2)}
            #append row to the dataframe
            colna_df = colna_df.append(new_row, ignore_index=True)
            
    colna_df= colna_df.sort_values("% of Null", ascending=False)    
    return colna_df  

# check rows for nulls
def check_rowna(df,supporting_fs):  
    """
    Check each row for nulls.Returns VID, total null and % of null for each row
    df[DataFrame]: dataframe
    supporting_fs[list]: all features not used for ML except VID and SPECKLE(target)
    """
    df = df.drop(supporting_fs,axis=1)    
    colrow_df = pd.DataFrame(columns =["SPECKLE","VID", "Total Null", "% of Null"])
    for i in df.index: 
        #checking if there is any null in the row
        if df.iloc[i].isnull().sum()>0:             
            # if null present, total number of null in the row stores here
            total_null = df.iloc[i].isnull().sum() 
            new_row = {'SPECKLE':df.iloc[i,0],'VID':df.iloc[i,1], 'Total Null':total_null, '% of Null':round(total_null*100/(len(df.columns)-2),2)}
            #append row to the dataframe
            colrow_df = colrow_df.append(new_row, ignore_index=True)
            
    colrow_df= colrow_df.sort_values("% of Null", ascending=False)    
    return colrow_df   

# Drop columns based on NA threshold limit
def drop_NAcol(df,NA_limit):
    '''
    Drops columns based on proportion of NA in column
    df[DataFrame]: df
    NA_limit[float/int]: Columns with proportion of NA above NA_limit will be dropped
    '''
    threshold = len(df)*(1-NA_limit)
    df=df.dropna(axis=1, thresh=threshold)
    print(df.shape)
    return df


#### 1) Check column for nulls

In [38]:
#check column na for whole df
colna_df =check_colna(df)
colna_df

,Feature,Total Null,% of Null
1230,SIO_LKG_X::HI1P05_X_LKG_K_END_X_X_VMAX_X_1MA_P...,16129,81.31
0,IDV_0001_SVT3GNES12_FULLDIE_SOC_0650_MED@132110,13847,69.81
399,IDV_0134_PSVT3GVTO12_FULLDIE_TALL_0950_MED@132110,13847,69.81
392,IDV_0134_PSVT3GVTO12_FULLDIE_CORE_TALL_0650_ME...,13847,69.81
393,IDV_0134_PSVT3GVTO12_FULLDIE_CORE_TALL_0950_ME...,13847,69.81
...,...,...,...
1472,SIO_LKG_X::HI1P05_X_LKG_K_END_X_X_VMAX_X_1MA_P...,13459,67.85
1473,SIO_LKG_X::HI1P05_X_LKG_K_END_X_X_VMAX_X_1MA_P...,13459,67.85
1407,SIO_LKG_X::HI1P05_X_LKG_K_END_X_X_VMAX_X_1MA_P...,13459,67.85
1406,SIO_LKG_X::HI1P05_X_LKG_K_END_X_X_VMAX_X_1MA_P...,13459,67.85


In [ ]:
colna_df.to_csv(path+"/DataPreparation/NA_Cols.csv",index=False)

In [39]:
#check column na for speckle
colna_speckle_df =check_colna(df.loc[df['SPECKLE'] == 1])
colna_speckle_df

,Feature,Total Null,% of Null
1230,SIO_LKG_X::HI1P05_X_LKG_K_END_X_X_VMAX_X_1MA_P...,5268,81.73
0,IDV_0001_SVT3GNES12_FULLDIE_SOC_0650_MED@132110,4845,75.16
399,IDV_0134_PSVT3GVTO12_FULLDIE_TALL_0950_MED@132110,4845,75.16
392,IDV_0134_PSVT3GVTO12_FULLDIE_CORE_TALL_0650_ME...,4845,75.16
393,IDV_0134_PSVT3GVTO12_FULLDIE_CORE_TALL_0950_ME...,4845,75.16
...,...,...,...
1472,SIO_LKG_X::HI1P05_X_LKG_K_END_X_X_VMAX_X_1MA_P...,4677,72.56
1473,SIO_LKG_X::HI1P05_X_LKG_K_END_X_X_VMAX_X_1MA_P...,4677,72.56
1406,SIO_LKG_X::HI1P05_X_LKG_K_END_X_X_VMAX_X_1MA_P...,4677,72.56
1405,SIO_LKG_X::HI1P05_X_LKG_K_END_X_X_VMAX_X_1MA_P...,4677,72.56


In [40]:
#check column na for non-speckle
colna_nonspeckle_df =check_colna(df.loc[df['SPECKLE'] == 0])
colna_nonspeckle_df

,Feature,Total Null,% of Null
1230,SIO_LKG_X::HI1P05_X_LKG_K_END_X_X_VMAX_X_1MA_P...,10861,81.11
875,TPI_SIU_STATIC::SIU_STATIC_AM_K_STRESS_X_X_X_S...,9006,67.26
882,TPI_SIU_STATIC::SIU_STATIC_AM_K_STRESS_X_X_X_S...,9006,67.26
889,TPI_SIU_STATIC::SIU_STATIC_AM_K_STRESS_X_X_X_S...,9006,67.26
888,TPI_SIU_STATIC::SIU_STATIC_AM_K_STRESS_X_X_X_S...,9006,67.26
...,...,...,...
820,TPI_SIU_STATIC::SIU_STATIC_AM_K_START_X_X_X_SN...,8782,65.59
822,TPI_SIU_STATIC::SIU_STATIC_AM_K_START_X_X_X_SN...,8782,65.59
824,TPI_SIU_STATIC::SIU_STATIC_AM_K_START_X_X_X_SN...,8782,65.59
826,TPI_SIU_STATIC::SIU_STATIC_AM_K_START_X_X_X_SN...,8782,65.59


#### 2) Check row for nulls

In [41]:
supporting_fs = ['ULT@MIDAS_6261_U1', 'SORTLOT', 'SORTLOT7', 'WAFER',
       'XLOC', 'YLOC', 'ULT@MIDAS_6261_U2', 'IB@6261[CLASSHOT]',
       'FB@6261[CLASSHOT]','ARR_SPECKLE_SCREEN::PHYTON_X_USERFUNC_K_END_X_X_X_X_PBISTSPECKLE_INDICATOR::DUTCPUBAD_1@6261[CLASSHOT]',
       'TEST RESULTS', 'TEST RESULTS BITS', 'TR_BITS', 'INCOMING',
       'INCOMING BITS', 'INC_BITS', 'OUTGOING', 'OUTGOING BITS', 'OUT_BITS','DELTA']
rowna_df = check_rowna(df,supporting_fs)
rowna_df

,SPECKLE,VID,Total Null,% of Null
8213,0,M1GC346401871,1593,100.00
10375,0,M11H9B4300835,1593,100.00
10377,0,M11H9B4301228,1593,100.00
10378,0,M11H9B4300032,1593,100.00
10379,0,M11H9B4301227,1593,100.00
...,...,...,...,...
11806,0,M05C0K5600161,1,0.06
11811,0,M0U25B1100396,1,0.06
11814,0,M0U25B1100907,1,0.06
11815,0,M0U25B1100798,1,0.06


In [42]:
rowna_df.loc[rowna_df["% of Null"] ==100.00]

,SPECKLE,VID,Total Null,% of Null
8213,0,M1GC346401871,1593,100.0
10375,0,M11H9B4300835,1593,100.0
10377,0,M11H9B4301228,1593,100.0
10378,0,M11H9B4300032,1593,100.0
10379,0,M11H9B4301227,1593,100.0
...,...,...,...,...
5463,0,M10P67G000951,1593,100.0
5464,0,M10P67G001238,1593,100.0
5468,0,M17U1C0300567,1593,100.0
5465,0,M10P67G000323,1593,100.0


In [ ]:
# rowna_df.to_csv(path+"/DataPreparation/NA_Rows_aftercolremoval.csv",index=False)

#### 3) Drop columns based on threshold limit

In [43]:
df = drop_NAcol(df,0.8) #drop columns with >80% NA 

(19836, 1614)


In [44]:
check_colna(df)

,Feature,Total Null,% of Null
0,IDV_0001_SVT3GNES12_FULLDIE_SOC_0650_MED@132110,13847,69.81
390,IDV_0133_PSVT3GVTO12_FULLDIE_TALL_0650_MED@132110,13847,69.81
392,IDV_0134_PSVT3GVTO12_FULLDIE_CORE_TALL_0650_ME...,13847,69.81
393,IDV_0134_PSVT3GVTO12_FULLDIE_CORE_TALL_0950_ME...,13847,69.81
394,IDV_0134_PSVT3GVTO12_FULLDIE_SOC_0650_MED@132110,13847,69.81
...,...,...,...
1471,SIO_LKG_X::HI1P05_X_LKG_K_END_X_X_VMAX_X_1MA_P...,13459,67.85
1472,SIO_LKG_X::HI1P05_X_LKG_K_END_X_X_VMAX_X_1MA_P...,13459,67.85
1406,SIO_LKG_X::HI1P05_X_LKG_K_END_X_X_VMAX_X_1MA_P...,13459,67.85
1405,SIO_LKG_X::HI1P05_X_LKG_K_END_X_X_VMAX_X_1MA_P...,13459,67.85


In [45]:
rowna_df = check_rowna(df,supporting_fs)
rowna_df

,SPECKLE,VID,Total Null,% of Null
7185,0,M1GC346401179,1592,100.00
9472,0,M1WE241501821,1592,100.00
9453,0,M1WE241501337,1592,100.00
9454,0,M1WE241501323,1592,100.00
9455,0,M1WE241500587,1592,100.00
...,...,...,...,...
10827,0,M12T24A201085,1,0.06
4973,0,M10P67G000982,1,0.06
303,1,M1FW102400277,1,0.06
6734,0,M1F84S6201738,1,0.06


In [46]:
rowna_df.loc[rowna_df["% of Null"] ==100.00]

,SPECKLE,VID,Total Null,% of Null
7185,0,M1GC346401179,1592,100.0
9472,0,M1WE241501821,1592,100.0
9453,0,M1WE241501337,1592,100.0
9454,0,M1WE241501323,1592,100.0
9455,0,M1WE241500587,1592,100.0
...,...,...,...,...
4743,1,M1NQ071700560,1592,100.0
4742,1,M1NQ071700139,1592,100.0
5124,0,M10P67G000855,1592,100.0
4741,1,M1NQ071700211,1592,100.0


In [ ]:
# df1[df1.isna().all(axis=1)]

In [ ]:
#drop NA rows
df = df.dropna(axis=0,how = "all")

In [ ]:
#NA imputation
def NA_impute(df,imptype):
    """
    Impute NA
    df[DataFrame]:df
    imptype[string]: "mean" to impute data with mean, "median" to impute data with median
    """
    if type == "mean":        
        df.fillna(df.mean())
    if type == "median":
        df.fillna(df.median())
    return df

### Handling unary columns

In [69]:
def unary(df):
    """
    Checks for unary columns. If there are unary columns, the unary columns will be printed and removed from the df
    df[DataFrame]: input dataframe
    """
    unarycolumns = [col for col in df.columns if len(df[col].unique())==1]
    if unarycolumns:
        print("The unary column are:",unarycolumns)
        df = df.drop(unarycolumns,axis=1)
        print("Unary columns dropped!")
        return df 
    else:
        print("There are no unary columns!")

In [70]:
unary(df)

There are no unary columns!


In [76]:
supporting_fs = ['SPECKLE','VID','ULT@MIDAS_6261_U1', 'SORTLOT', 'SORTLOT7', 'WAFER',
       'XLOC', 'YLOC', 'ULT@MIDAS_6261_U2', 'IB@6261[CLASSHOT]',
       'FB@6261[CLASSHOT]','ARR_SPECKLE_SCREEN::PHYTON_X_USERFUNC_K_END_X_X_X_X_PBISTSPECKLE_INDICATOR::DUTCPUBAD_1@6261[CLASSHOT]',
       'TEST RESULTS', 'TEST RESULTS BITS', 'TR_BITS', 'INCOMING',
       'INCOMING BITS', 'INC_BITS', 'OUTGOING', 'OUTGOING BITS', 'OUT_BITS','DELTA']
df1 = df.drop(supporting_fs,axis=1) 
df1.head()

,IDV_0001_SVT3GNES12_FULLDIE_SOC_0650_MED@132110,IDV_0001_SVT3GNES12_FULLDIE_SOC_0950_MED@132110,IDV_0001_SVT3GNES12_FULLDIE_SOC_TALL_0650_MED@132110,IDV_0001_SVT3GNES12_FULLDIE_SOC_TALL_0950_MED@132110,IDV_0002_SVT3GNES12_FULLDIE_SOC_0650_MED@132110,IDV_0002_SVT3GNES12_FULLDIE_SOC_0950_MED@132110,IDV_0002_SVT3GNES12_FULLDIE_SOC_TALL_0650_MED@132110,IDV_0002_SVT3GNES12_FULLDIE_SOC_TALL_0950_MED@132110,IDV_0003_SVT3GSEM12_FULLDIE_SOC_0650_MED@132110,IDV_0003_SVT3GSEM12_FULLDIE_SOC_0950_MED@132110,...,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_SVIDCLK1,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_SVIDDATA0,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_SVIDDATA1,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TAP_DIS_STRAP_N,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TAP_ODT_EN,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TXT_AGENT,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TXT_PLTEN,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_VPPSMBUSSCL,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_VPPSMBUSSDA,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_VSENSEPMAX
0,6016.90,12304.2,5791.55,12112.7,5994.37,12293.0,5887.32,12214.1,5245.07,11600.0,...,-8.210000e-07,-8.210000e-07,-8.330000e-07,-2.100000e-07,-2.240000e-07,-1.950000e-07,-1.910000e-07,-8.990000e-07,-8.840000e-07,-1.970000e-07
1,6073.24,12214.1,6005.63,12163.4,6078.87,12174.7,6061.97,12236.6,5295.77,11560.6,...,-8.670000e-07,-8.870000e-07,-8.740000e-07,-2.110000e-07,-2.100000e-07,-1.900000e-07,-2.280000e-07,-8.620000e-07,-8.610000e-07,-1.920000e-07
2,6343.66,12231.0,6163.38,12028.2,6264.79,12152.1,6219.72,12129.6,5695.77,11639.4,...,-6.000000e-07,-6.500000e-07,-6.380000e-07,-1.440000e-07,-1.380000e-07,-1.440000e-07,-1.620000e-07,-5.820000e-07,-6.150000e-07,-1.460000e-07
3,5566.20,11453.5,5442.25,11357.8,5554.93,11414.1,5476.06,11391.5,4811.27,10777.5,...,-1.170000e-06,-1.220000e-06,-1.230000e-06,-2.950000e-07,-2.930000e-07,-3.190000e-07,-3.070000e-07,-1.220000e-06,-1.190000e-06,-2.960000e-07
4,6247.89,12293.0,6112.68,12180.3,6219.72,12270.4,6163.38,12259.2,5470.42,11639.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Train-validation split

In [70]:
from sklearn.model_selection import train_test_split
def randomsamp(df,val_size):
    """
    Split whole dataset into train and validation using random sampling
    Returns X_train, X_val, y_train, y_val
    df[DataFrame]: input dataframe    
    val_size[float]:Should be between 0.0 and 1.0 and represent the proportion of the dataset to include in the val split. 
                    Train size is complement of val size
    """
    X= df.drop(["SPECKLE"],axis=1)
    y= df["SPECKLE"] 
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = val_size,random_state=42,stratify= df["SPECKLE"])
    if type(val_size)==float:
        print("Train-val split completed with",(1-val_size)*100,"-",val_size*100,"split in train-val")
    print("Shape of X_train is:", X_train.shape)
    print("Shape of X_val is:",X_val.shape)
    print("Shape of y_train is:",y_train.shape)
    print("Shape of y_val is:",y_val.shape)
    print("Distribution of y_train:",Counter(y_train))
    print("Distribution of y_val:",Counter(y_val))
    
    return X_train, X_val, y_train, y_val

def passfailsamp(df,speckle_test_size,nonspeckle_test_size):
    """
    Dataset is split into speckle/non-speckle first. The speckle/non-speckle datasets are then split 
    into train and validation using random sampling, followed by merging of the speckle/non-speckle  
    to return X_train, X_val, y_train, y_val
    
    df[DataFrame]: input dataframe  
    
    speckle_test_size[float/int]: Val size for speckle 
    -If float, should be between 0.0 and 1.0 and represent the proportion of the dataset to include in the val split. 
    -If int, represents the absolute number of val samples.
    -Train size is complement of val size
    
    nonspeckle_test_size[float/int]: Val size for non-speckle 
    -If float, should be between 0.0 and 1.0 and represent the proportion of the dataset to include in the val split. 
    -If int, represents the absolute number of val samples.
    -Train size is complement of val size
    
    """
    #split data into speckle/nonspeckle
    df_s = df.loc[df["SPECKLE"] == 1] #speckle
    df_ns = df.loc[df["SPECKLE"] == 0] #nonspeckle
    
    #split speckle/nonspeckle data into train/validation
    print("For speckle data:")
    X_train_s, X_val_s, y_train_s, y_val_s = randomsamp(df_s,val_size=speckle_test_size)
    print("\nFor non-speckle data:")
    X_train_ns, X_val_ns, y_train_ns, y_val_ns = randomsamp(df_ns,val_size=nonspeckle_test_size)
    
    #concat the speckle/non-speckle train and validation
    X_train = pd.concat([X_train_s,X_train_ns], ignore_index=True)
    X_val = pd.concat([X_val_s,X_val_ns], ignore_index=True)
    y_train = pd.concat([y_train_s,y_train_ns], ignore_index=True)
    y_val = pd.concat([y_val_s,y_val_ns], ignore_index=True)
    
    print("\nFinal dataset:")
    print("Distribution of y_train:",Counter(y_train))
    print("Distribution of y_val:",Counter(y_val))
    return X_train, X_val, y_train, y_val

In [76]:
df["SPECKLE"].value_counts()

0    13390
1     6446
Name: SPECKLE, dtype: int64

In [71]:
#train-test split on whole data
X_train, X_val, y_train, y_val = randomsamp(df,val_size=0.3)

Train-val split completed with 70.0 - 30.0 split in train-val
Shape of X_train is: (13885, 1614)
Shape of X_val is: (5951, 1614)
Shape of y_train is: (13885,)
Shape of y_val is: (5951,)
Distribution of y_train: Counter({0: 9373, 1: 4512})
Distribution of y_val: Counter({0: 4017, 1: 1934})


In [75]:
#train-test split on speckle/non-speckle then merge
X_train, X_val, y_train, y_val = passfailsamp(df,speckle_test_size=500,nonspeckle_test_size=8000)

For speckle data:
Shape of X_train is: (5946, 1614)
Shape of X_val is: (500, 1614)
Shape of y_train is: (5946,)
Shape of y_val is: (500,)
Distribution of y_train: Counter({1: 5946})
Distribution of y_val: Counter({1: 500})

For non-speckle data:
Shape of X_train is: (5390, 1614)
Shape of X_val is: (8000, 1614)
Shape of y_train is: (5390,)
Shape of y_val is: (8000,)
Distribution of y_train: Counter({0: 5390})
Distribution of y_val: Counter({0: 8000})

Final dataset:
Distribution of y_train: Counter({1: 5946, 0: 5390})
Distribution of y_val: Counter({0: 8000, 1: 500})


In [84]:
# num = df._get_numeric_data()
# num.head()

,SPECKLE,WAFER,XLOC,YLOC,IB@6261[CLASSHOT],FB@6261[CLASSHOT],TEST RESULTS BITS,TR_BITS,INCOMING BITS,INC_BITS,...,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_SVIDCLK1,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_SVIDDATA0,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_SVIDDATA1,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TAP_DIS_STRAP_N,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TAP_ODT_EN,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TXT_AGENT,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TXT_PLTEN,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_VPPSMBUSSCL,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_VPPSMBUSSDA,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_VSENSEPMAX
0,1,293,6,-3,1,101,10,1,0,0,...,-8.210000e-07,-8.210000e-07,-8.330000e-07,-2.100000e-07,-2.240000e-07,-1.950000e-07,-1.910000e-07,-8.990000e-07,-8.840000e-07,-1.970000e-07
1,1,225,5,-5,1,101,10100,2,100,1,...,-8.670000e-07,-8.870000e-07,-8.740000e-07,-2.110000e-07,-2.100000e-07,-1.900000e-07,-2.280000e-07,-8.620000e-07,-8.610000e-07,-1.920000e-07
2,1,135,6,0,1,101,100000,1,0,0,...,-6.000000e-07,-6.500000e-07,-6.380000e-07,-1.440000e-07,-1.380000e-07,-1.440000e-07,-1.620000e-07,-5.820000e-07,-6.150000e-07,-1.460000e-07
3,1,159,3,1,1,101,101000,2,1000,1,...,-1.170000e-06,-1.220000e-06,-1.230000e-06,-2.950000e-07,-2.930000e-07,-3.190000e-07,-3.070000e-07,-1.220000e-06,-1.190000e-06,-2.960000e-07
4,1,45,1,5,1,101,101000,2,1000,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [82]:
# (num < 0).any()

SPECKLE                                                                False
WAFER                                                                  False
XLOC                                                                    True
YLOC                                                                    True
IB@6261[CLASSHOT]                                                      False
                                                                       ...  
SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TXT_AGENT       True
SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TXT_PLTEN       True
SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_VPPSMBUSSCL     True
SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_VPPSMBUSSDA     True
SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_VSENSEPMAX      True
Length: 1604, dtype: bool

In [86]:
# num.loc[:,(num < 0).any()]

,XLOC,YLOC,IDV_0101_SVT3GNES12_FULLDIE_CORE_TALL_0650_MED@132110,IDV_0102_SVT3GNES12_FULLDIE_CORE_TALL_0650_MED@132110,IDV_0103_SVT3GSEM12_FULLDIE_CORE_TALL_0650_MED@132110,IDV_0104_NOM3GNES12_FULLDIE_CORE_TALL_0650_MED@132110,IDV_0105_NOM3GNES12_FULLDIE_CORE_TALL_0650_MED@132110,IDV_0106_SVT3GISO12_FULLDIE_CORE_TALL_0650_MED@132110,IDV_0107_SVT1GNES12_FULLDIE_CORE_TALL_0650_MED@132110,IDV_0108_SVT1GNES12_FULLDIE_CORE_TALL_0650_MED@132110,...,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_SVIDCLK1,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_SVIDDATA0,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_SVIDDATA1,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TAP_DIS_STRAP_N,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TAP_ODT_EN,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TXT_AGENT,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_TXT_PLTEN,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_VPPSMBUSSCL,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_VPPSMBUSSDA,SIO_LKG_X::LO1P05_X_LKG_K_END_X_X_VMAX_X_1MA_PC1@132110_VSENSEPMAX
0,6,-3,5746.48,5780.28,5059.15,7616.90,7661.97,4061.97,3656.34,3718.31,...,-8.210000e-07,-8.210000e-07,-8.330000e-07,-2.100000e-07,-2.240000e-07,-1.950000e-07,-1.910000e-07,-8.990000e-07,-8.840000e-07,-1.970000e-07
1,5,-5,6005.63,6084.51,5273.24,7977.46,8045.07,4292.96,3977.46,4000.00,...,-8.670000e-07,-8.870000e-07,-8.740000e-07,-2.110000e-07,-2.100000e-07,-1.900000e-07,-2.280000e-07,-8.620000e-07,-8.610000e-07,-1.920000e-07
2,6,0,6208.45,6242.25,5684.51,7616.90,7650.70,4760.56,3932.39,3977.46,...,-6.000000e-07,-6.500000e-07,-6.380000e-07,-1.440000e-07,-1.380000e-07,-1.440000e-07,-1.620000e-07,-5.820000e-07,-6.150000e-07,-1.460000e-07
3,3,1,5430.99,5442.25,4659.15,7076.06,7109.86,3707.04,3459.15,3492.96,...,-1.170000e-06,-1.220000e-06,-1.230000e-06,-2.950000e-07,-2.930000e-07,-3.190000e-07,-3.070000e-07,-1.220000e-06,-1.190000e-06,-2.960000e-07
4,1,5,6061.97,6123.94,5380.28,7864.79,7909.86,4394.37,3966.20,3994.37,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19831,2,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19832,1,-6,5402.82,5453.52,4783.10,7008.45,7030.99,3921.13,3656.34,3707.04,...,-1.060000e-06,-1.120000e-06,-1.130000e-06,-2.610000e-07,-2.700000e-07,-2.700000e-07,-2.720000e-07,-1.090000e-06,-1.110000e-06,-2.270000e-07
19833,3,-5,5960.56,6028.17,5256.34,7819.72,7864.79,4281.69,3876.06,3909.86,...,-9.820000e-07,-1.010000e-06,-1.010000e-06,-2.290000e-07,-2.640000e-07,-2.570000e-07,-2.450000e-07,-9.540000e-07,-9.890000e-07,-2.640000e-07
19834,4,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# df['AbsAmount'] = df['Amount'].abs()

In [ ]:
#create new column for target
# import numpy as np
# df["SPECKLE"] = np.where(df["DELTA"]==0,0,1)
# cols = df.columns.tolist()
# cols = [cols[-1]]+cols[:-1] #move speckle col to the front
# df = df.reindex(columns=cols)

In [ ]:
# df.to_csv(path+"SNR_R2_ww35.2_Speckle.csv",index=False)

In [ ]:
# supporting_fs = ['ULT@MIDAS_6261_U1', 'SORTLOT', 'SORTLOT7', 'WAFER',
#        'XLOC', 'YLOC', 'ULT@MIDAS_6261_U2', 'IB@6261[CLASSHOT]',
#        'FB@6261[CLASSHOT]','ARR_SPECKLE_SCREEN::PHYTON_X_USERFUNC_K_END_X_X_X_X_PBISTSPECKLE_INDICATOR::DUTCPUBAD_1@6261[CLASSHOT]',
#        'TEST RESULTS', 'TEST RESULTS BITS', 'TR_BITS', 'INCOMING',
#        'INCOMING BITS', 'INC_BITS', 'OUTGOING', 'OUTGOING BITS', 'OUT_BITS','DELTA']
# df1 = df.drop(supporting_fs,axis=1) 
# df1.head()

### Feature Selection

### Model Building